# Workshop02 : Réseaux de neurones (Auto-encodeurs et CNN)

In [ ]:
import os
import re
import math
from pathlib import Path

In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
import matplotlib.pyplot as plt 
import matplotlib.image  as mpimg
from   matplotlib        import colors

%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Lambda, Flatten, Reshape, Dropout
from tensorflow.keras.layers import Conv2D, MaxPool2D, AvgPool2D, Dropout, UpSampling2D
from tensorflow.keras.models import Sequential, Model

tf.__version__

## Pré-traitement

In [ ]:
# Lorsqu'on lit l'image PNG avec 3 canaux, 
# les parties transparentes sont transformées en noir 
# Or, normalement elle doivent être en blanc
# Donc, j'ai supposé que la partie transparente est encodée sous forme (0, 0, 0, 0)
# Dans ce cas, on la transforme à (255, 255, 255)
def rgb_vers_rgba(img):
    h, w, c = img.shape
    if c != 4:
        return img
    
    resh_img = img.reshape([-1, c])
    new_img = []
    
    for i in range(len(resh_img)):
        l = resh_img[i, :]
        if l[3] == 0:
            new_img.append([255, 255, 255])
        else:
            new_img.append(l[:-1])
    return np.array(new_img).reshape([h, w, 3])
    
rgb_vers_rgba(np.array([
    [[255, 200, 100, 255], [255, 200, 100, 255], [255, 200, 100, 0]],
    [[255, 200, 100, 255], [0, 0, 0, 0], [255, 200, 100, 200]]
]))

In [ ]:
DATASET_LOC = 'pokemon/'

types_df = pd.read_csv(DATASET_LOC + 'pokemon.csv')
types_df = types_df.drop(['Type2'], axis=1)

def generer_dataset(ext='*.png'):
    d = DATASET_LOC + 'images/images'
    path = Path(d)
    fichiers = list(path.glob(ext))
    noms = [os.path.split(fichier)[1] for fichier in fichiers ]
    image_df = pd.concat([pd.Series(noms, name='Name'), pd.Series(fichiers, name='path').astype(str)], axis=1)
    image_df['Name'] = image_df['Name'].apply(lambda x: re.sub(r'\.\w+$', '', x))
    
    df = image_df.merge(types_df, on='Name')
    
    imgs = []
    for index in range(df.shape[0]):
        img = tf.io.decode_image(tf.io.read_file(df['path'][index]), channels=0)
        img = rgb_vers_rgba(img.numpy())
        imgs.append(img)
#         imgs.append(mpimg.imread(df['path'][index]))
    
    return np.array(imgs), df['Type1'].values

def afficher_images(imgs, y, n):
    plt.figure(figsize=(20, 4))
    for i in range(n):
        # display original
        ax = plt.subplot(1, n, i + 1)
        plt.imshow(imgs[i]) # recontruct the matrix
        plt.gray()
        plt.title("T: " + str(y[i]))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()
    
def classer_images(imgs, cls, cls_mdl, n):
    plt.figure(figsize=(20, 4))
    Y_pred = cls_mdl.predict(imgs[0:n])
    for i in range(n):
        # display original
        ax = plt.subplot(1, n, i + 1)
        plt.imshow(imgs[i]) # recontruct the matrix
        plt.gray()
        plt.title("T: " + str(cls[i]) + ', P: ' + str(Y_pred[i]))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()
    
print('Fonction definie')

In [ ]:
# dataset d'entraînement
X_train, Y_train = generer_dataset()

X_train = X_train / 255

# 721 images de 120X120 pixels chaque pixel est encodé sur 3 canaux 
print(X_train.shape)

afficher_images(X_train, Y_train, 10)

In [ ]:
# dataset de test
X_test, Y_test = generer_dataset('*.jpg')

X_test = X_test / 255

# 721 images de 120X120 pixels chaque pixel est encodé sur 4 canaux 
print(X_test.shape)

afficher_images(X_test, Y_test, 10)

In [ ]:
from sklearn.preprocessing import LabelEncoder

pokemon_type_enc = LabelEncoder()

Y_train = pokemon_type_enc.fit_transform(Y_train)
Y_test = pokemon_type_enc.transform(Y_test)

cls_len = len(np.unique(Y_train))

Y_train_onehot = tf.keras.utils.to_categorical(Y_train, cls_len)
Y_test_onehot = tf.keras.utils.to_categorical(Y_test, cls_len)

# Le dataset de test n'est pas vraiment représentatif : les classes 7 et 11 
# ne figurent pas dans le test
np.unique(Y_train), np.unique(Y_test)

## 1. Classement CNN

## 2. Auto-encodeur de clustering

## 3. Auto-encodeur de clustering pour classement

## 4. Génération des images